In [1]:
import pandas as pd
import reverse_geocoder as rg
from geopy.geocoders import Nominatim 
from googletrans import Translator, constants
translator = Translator()
geolocator = Nominatim(user_agent="geoapiExercises")

In [47]:
e_data_with_country = pd.read_csv('out_e_data.csv')

In [2]:
e_data = pd.read_csv("https://www.dropbox.com/s/u2xupuhf07y85zh/earthQuake.csv?dl=1")
density_data = pd.read_csv("https://www.dropbox.com/s/p6ejr4mzx8pjq9m/population_density_long.csv?dl=1")
population_data=pd.read_csv("https://www.dropbox.com/s/bp5hb61zrui2qo6/population_total_long.csv?dl=1")

separate the date into columns

In [3]:
e_data[['Month','Day','Year']]=e_data['Date'].str.split('/',-1,expand=True)

get rid of all unwanted columns (is Depth usefull?)

In [4]:
e_data = e_data.drop(['Date','Time','ID','Source','Location Source','Magnitude Source','Status','Type','Magnitude Type'],axis=1)

In [5]:
print(e_data)

       Latitude  Longitude   Depth  Magnitude Month Day  Year
0       19.2460   145.6160  131.60        6.0    01  02  1965
1        1.8630   127.3520   80.00        5.8    01  04  1965
2      -20.5790  -173.9720   20.00        6.2    01  05  1965
3      -59.0760   -23.5570   15.00        5.8    01  08  1965
4       11.9380   126.4270   15.00        5.8    01  09  1965
...         ...        ...     ...        ...   ...  ..   ...
23407   38.3917  -118.8941   12.30        5.6    12  28  2016
23408   38.3777  -118.8957    8.80        5.5    12  28  2016
23409   36.9179   140.4262   10.00        5.9    12  28  2016
23410   -9.0283   118.6639   79.00        6.3    12  29  2016
23411   37.3973   141.4103   11.94        5.5    12  30  2016

[23412 rows x 7 columns]


In [6]:
population_data = population_data.drop(population_data.index[population_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
population_data.rename(columns={'Count': 'Population'}, inplace=True)

In [7]:
density_data = density_data.drop(density_data.index[density_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
density_data.rename(columns={'Count': 'Density'}, inplace=True)

In [31]:
def find_country(long,lat):
    tuple_input = (long,lat)
    found = rg.search(tuple_input)
    countryName = found[0]['name']
#     countryName = translator.translate(countryName)
    print('**', countryName)
    
    return countryName

def city_to_country(city):
    try:
        loc = geolocator.geocode(city, language='en')
        loc_list = loc.address.split(',')
    except:
        print(city + ': Not found')
        return None
    return loc_list[-1]
    

In [9]:
joined = density_data.merge(population_data,
          how='left',
          on =['Country Name', 'Year'],
         )
joined['Population'] = joined['Population'].fillna(0).astype(int)
print(joined)
pop_dense_data = joined
#  suffixes=['', '_r']).drop(columns = ['date_r', 'year-month']

       Country Name  Year  Density  Population
0             Aruba  1965      318       57360
1       Afghanistan  1965       15     9956320
2            Angola  1965        4     5770570
3           Albania  1965       68     1864791
4           Andorra  1965       39       18549
...             ...   ...      ...         ...
11230        Kosovo  2017      168     1830700
11231   Yemen, Rep.  2017       52    27834821
11232  South Africa  2017       46    57000451
11233        Zambia  2017       22    16853688
11234      Zimbabwe  2017       36    14236745

[11235 rows x 4 columns]


In [43]:
count = 0
for i, r, in e_data.iterrows():
    country = city_to_country(find_country(r['Latitude'], r['Longitude']))
    e_data['Country Name'][count] = country
    print(country)
    count +=1
    

** Saipan
 United States


<ipython-input-43-8fddddeffdf5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_data['Country Name'][count] = country


** Tongutisungi
Tongutisungi: Not found
None
** Pangai
 Tonga
** Grytviken
 South Georgia and the South Sandwich Islands
** Dapdap
 Philippines
** Sola
 Norway
** Naya Bazar
 Nepal
** Sola
 Norway
** Grytviken
 South Georgia and the South Sandwich Islands
** Suva
 Fiji
** Bulakamba
 Indonesia
** Sanana
 Indonesia
** Ust'-Kamchatsk Staryy
Ust'-Kamchatsk Staryy: Not found
None
** Nuku'alofa
 Tonga
** Murghob
 Tajikistan
** Geeveston
 Australia
** Beringovskiy
 Russia
** Beringovskiy
 Russia
** Tilichiki
 Russia
** Beringovskiy
 Russia
** Beringovskiy
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Beringovskiy
 Russia
** Tilichiki
 Russia
** Beringovskiy
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Beringovskiy
 Russia
** Remire-Montjoly
 France
** Tilichiki
 Russia
** Beringovskiy
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Tilichiki
 Russia
** Beringovskiy
 Russia
** Sand Point
 United States
** Bering

** Haines Junction
 Canada
** Alo
 France
** Kokopo
 Papua New Guinea
** Tateyama
 Japan
** Akutan
 United States
** Puquina
 Bolivia
** Dawei
 Myanmar
** Ogotua
 Indonesia
** Toba
 Japan
** Tilichiki
 Russia
** Tilichiki
 Russia
** Ryotsu-minato
Ryotsu-minato: Not found
None
** Kodiak Station
 United States
** Barkhan
 Pakistan
** Grytviken
 South Georgia and the South Sandwich Islands
** Port-Vila
 Vanuatu
** Ozernovskiy
 Russia
** Las Lajitas
 Argentina
** Bilungala
 Indonesia
** Binabalian
 Philippines
** Hualian
 Taiwan
** Lakatoro
 Vanuatu
** Kerasochori
 Greece
** Jinzhong
 China
** Severo-Kuril'sk
 Russia
** Barkhan
 Pakistan
** Musa Khel Bazar
 Pakistan
** Grytviken
 South Georgia and the South Sandwich Islands
** Sekongkang Bawah
 Indonesia
** Katsuura
 Japan
** Saipan
 United States
** Tual
 Indonesia
** Jinzhong
 China
** Port-Vila
 Vanuatu
** Port-aux-Francais
 France
** Port-aux-Francais
 France
** Taguisa
 Philippines
** Grytviken
 South Georgia and the South Sandwich Is

** Kirakira
 Solomon Islands
** Essang
 Indonesia
** Gul'cha
 Kyrgyzstan
** Iquique
 Chile
** Ushuaia
 Argentina
** Tanjunggadang
Tanjunggadang: Not found
None
** Grytviken
 South Georgia and the South Sandwich Islands
** Beringovskiy
 Russia
** Thang
 Laos
** Merizo Village
 United States
** Akutan
 United States
** Panguna
 Papua New Guinea
** Kodiak
 United States
** Ancud
 Chile
** East London
 South Africa
** Severo-Kuril'sk
 Russia
** Marcabamba
 Peru
** Hihifo
 Tonga
** Severo-Kuril'sk
 Russia
** Grytviken
 South Georgia and the South Sandwich Islands
** Unalaska
 United States
** Unalaska
 United States
** Taisha
 Ecuador
** Fairbanks
 United States
** Taltal
 Chile
** Geser
 Indonesia
** Port-Vila
 Vanuatu
** Fais
 Federated States of Micronesia
** Fais
 Federated States of Micronesia
** Gizo
 Egypt
** Riverton
 United States
** Lospalos
 East Timor
** Sand Point
 United States
** Port Blair
 India
** Shimo-furano
 Japan
** Unalaska
 United States
** Patin-ay
 Philippines
** E

...

** Isangel
 Vanuatu
** Augusta
 United States
** Augusta
 United States
** Hasaki
 Japan
** Merizo Village
 United States
** Adamstown
 Pitcairn Islands
** Okurcalar
 Turkey
** Georgetown
 Guyana
** Neiafu
 Tonga
** Saipan
 United States
** Buka
 Papua New Guinea
** Bitung
 Indonesia
** Burgos
 Spain
** Sola
 Norway
** Iquique
 Chile
** Namie
 Japan
** Novita
 Colombia
** Puerto Villamil
 Ecuador
** Iquique
 Chile
** Lata
 France
** `Ohonua
 Tonga
** Pajapita
 Guatemala
** Hatillo
 Costa Rica
** Dutch Harbor
 United States
** Masterton
 New Zealand
** Hihifo
 Tonga
** Hihifo
 Tonga
** Isangel
 Vanuatu
** Congkar
 Indonesia
** Yigo Village
 Taiwan
** Yigo Village
 Taiwan
** Adipala
 Indonesia
** Lixouri
 Greece
** Sami
 Greece
** Padilla
 Mexico
** Taro'a
 Thailand
** Putre
 Chile
** Grytviken
 South Georgia and the South Sandwich Islands
** Lata
 France
** Waitangi
 New Zealand
** Lixouri
 Greece
** Buka
 Papua New Guinea
** Lospalos
 East Timor
** Kettering
 United States
** Isangel
 

** Abasolo del Valle
 Mexico
** Lorengau
 Papua New Guinea
** Ribeira Grande
 Portugal
** Panguna
 Papua New Guinea
** Port Blair
 India
** Bordj el Kiffan
 Algeria
** Hithadhoo
 Maldives
** Grytviken
 South Georgia and the South Sandwich Islands
** Lorengau
 Papua New Guinea
** Lorengau
 Papua New Guinea
** Nago
 Japan
** Daliyuan
 China
** Lospalos
 East Timor
** Misawa
 Japan
** Pondaguitan
Pondaguitan: Not found
None
** Santa Rosalia
 Mexico
** `Ohonua
 Tonga
** Ancud
 Chile
** Yigo Village
 Taiwan
** Angoram
 Papua New Guinea
** Abdanan
 Iran
** Abdanan
 Iran
** Dehloran
 Iran
** Dehloran
 Iran
** Beringovskiy
 Russia
** Grytviken
 South Georgia and the South Sandwich Islands
** Kimbe
 Papua New Guinea
** Lae
 Papua New Guinea
** Iquique
 Chile
** Hacienda La Calera
 Mexico
** American Canyon
 United States
** Grytviken
 South Georgia and the South Sandwich Islands
** Upahuacho
 Peru
** Urasqui
 Peru
** Alo
 France
** Port-Olry
 Vanuatu
** Takanabe
 Japan
** Gefyra
 Greece
** Port

** El Palmarcito
 Bolivia
** Katsuura
 Japan
** Kodari
 Nepal
** Camgyai
Camgyai: Not found
None
** Ramechhap
 Nepal
** Ofunato
 Japan
** Rantaupangiang
Rantaupangiang: Not found
None
** Ramechhap
 Nepal
** Lata
 France
** Port-aux-Francais
 France
** Panguna
 Papua New Guinea
** Port-Vila
 Vanuatu
** Adamstown
 Pitcairn Islands
** Pangai
 Tonga
** Lata
 France
** Honiara
 Solomon Islands
** Kirakira
 Solomon Islands
** Kirakira
 Solomon Islands
** Kokopo
 Papua New Guinea
** Jamestown
 United States
** Pangai
 Tonga
** Grytviken
 South Georgia and the South Sandwich Islands
** Calama
 Chile
** Korido
 Indonesia
** Grytviken
 South Georgia and the South Sandwich Islands
** Dillingham
 United States
** Augusta
 United States
** Shimoda
 Japan
** Hihifo
 Tonga
** Katsuura
 Japan
** Bandon
 Indonesia
** Bandon
 Indonesia
** Ranau
 Malaysia
** Port-aux-Francais
 France
** Shizunai-furukawacho
Shizunai-furukawacho: Not found
None
** Adamstown
 Pitcairn Islands
** Miyako
 Japan
** Calama
 Ch

** Anchor Point
 Canada
** Pangai
 Tonga
** Al Hoceima
 Morocco
** Tobelo
 Indonesia
** Kokopo
 Papua New Guinea
** Kokopo
 Papua New Guinea
** Fernando de Noronha (Distrito Estadual)
 Brazil
** Yelizovo
 Russia
** Lorengau
 Papua New Guinea
** Lorengau
 Papua New Guinea
** McMurdo Station
 McMurdo Station
** `Ohonua
 Tonga
** Yonakuni
Yonakuni: Not found
None
** Yujing
 Taiwan
** Lautoka
 Fiji
** Hihifo
 Tonga
** Panguna
 Papua New Guinea
** Ovalle
 Chile
** Wailiang
 Indonesia
** Christchurch
 New Zealand
** Oyama
 Japan
** Haveluloto
 Tonga
** Riverton
 United States
** Atico
 Peru
** Adamstown
 Pitcairn Islands
** Petani
 Indonesia
** Grytviken
 South Georgia and the South Sandwich Islands
** Haveluloto
 Tonga
** Fernando de Noronha (Distrito Estadual)
 Brazil
** Saipan
 United States
** Coquimbo
 Chile
** Geeveston
 Australia
** McMurdo Station
 McMurdo Station
** Geeveston
 Australia
** Sarangani
 Philippines
** Waitangi
 New Zealand
** Muara Siberut
 Indonesia
** Muara Siberut
 

** Puerto Villamil
 Ecuador
** Ostrov Paskhi
Ostrov Paskhi: Not found
None
** Opotiki
 New Zealand
** Tongutisungi
Tongutisungi: Not found
None
** Hihifo
 Tonga
** Buka
 Papua New Guinea
** Ostrov Paskhi
Ostrov Paskhi: Not found
None
** Kimbe
 Papua New Guinea
** Asprangeloi
 Greece
** Kandrian
 Papua New Guinea
** Aduo
 China
** Haveluloto
 Tonga
** Pointe Michel
 Dominica
** Pamanukan
 Indonesia
** Remire-Montjoly
 France
** Saipan
 United States
** Kurayoshi
 Japan
** Kuril'sk
 Russia
** Manzanillo
 Spain
** Ostrov Paskhi
Ostrov Paskhi: Not found
None
** Hihifo
 Tonga
** Hihifo
 Tonga
** Preci
 Italy
** Visso
 Italy
** Georgetown
 Guyana
** Bitung
 Indonesia
** Akutan
 United States
** El Tabo
 Chile
** Kokopo
 Papua New Guinea
** Ustica
 Italy
** Preci
 Italy
** Colombia
Colombia
** Kandrian
 Papua New Guinea
** Curico
 Chile
** Tugu Hilir
 Indonesia
** Cygnet
 United States
** Talcahuano
 Chile
** Kimbe
 Papua New Guinea
** Denmark
Denmark
** Haveluloto
 Tonga
** Onagawa Cho
 Japa

In [ ]:
tuple_input = ('19','145')
found = rg.search(tuple_input)
countryName = found[0]['name']
print(found)

In [30]:
loc = geolocator.geocode('Russia', language='en')
print(loc)

Russia


In [33]:
# g = density_data.loc[(pop_dense_data['Country Name'] == countryName) & (pop_dense_data['Year'] == year)]
firstRun = pop_dense_data.loc[(pop_dense_data['Country Name'] == countryName)]
secondRun = firstRun.loc[(firstRun['Year'] == year)]

print(secondRun)

Empty DataFrame
Columns: [Country Name, Year, Density, Population]
Index: []


In [45]:

ee_data = e_data
print(e_data)


       Latitude  Longitude   Depth  Magnitude Month Day  Year  \
0       19.2460   145.6160  131.60        6.0    01  02  1965   
1        1.8630   127.3520   80.00        5.8    01  04  1965   
2      -20.5790  -173.9720   20.00        6.2    01  05  1965   
3      -59.0760   -23.5570   15.00        5.8    01  08  1965   
4       11.9380   126.4270   15.00        5.8    01  09  1965   
...         ...        ...     ...        ...   ...  ..   ...   
23407   38.3917  -118.8941   12.30        5.6    12  28  2016   
23408   38.3777  -118.8957    8.80        5.5    12  28  2016   
23409   36.9179   140.4262   10.00        5.9    12  28  2016   
23410   -9.0283   118.6639   79.00        6.3    12  29  2016   
23411   37.3973   141.4103   11.94        5.5    12  30  2016   

                                        Country Name  
0                                      United States  
1                                               None  
2                                              Tonga  

In [46]:
ee_data.to_csv('out_e_data.csv')

In [48]:
print(pop_dense_data)

       Country Name  Year  Density  Population
0             Aruba  1965      318       57360
1       Afghanistan  1965       15     9956320
2            Angola  1965        4     5770570
3           Albania  1965       68     1864791
4           Andorra  1965       39       18549
...             ...   ...      ...         ...
11230        Kosovo  2017      168     1830700
11231   Yemen, Rep.  2017       52    27834821
11232  South Africa  2017       46    57000451
11233        Zambia  2017       22    16853688
11234      Zimbabwe  2017       36    14236745

[11235 rows x 4 columns]


In [51]:
print(ee_data)
print()

       Latitude  Longitude   Depth  Magnitude Month Day  Year  \
0       19.2460   145.6160  131.60        6.0    01  02  1965   
1        1.8630   127.3520   80.00        5.8    01  04  1965   
2      -20.5790  -173.9720   20.00        6.2    01  05  1965   
3      -59.0760   -23.5570   15.00        5.8    01  08  1965   
4       11.9380   126.4270   15.00        5.8    01  09  1965   
...         ...        ...     ...        ...   ...  ..   ...   
23407   38.3917  -118.8941   12.30        5.6    12  28  2016   
23408   38.3777  -118.8957    8.80        5.5    12  28  2016   
23409   36.9179   140.4262   10.00        5.9    12  28  2016   
23410   -9.0283   118.6639   79.00        6.3    12  29  2016   
23411   37.3973   141.4103   11.94        5.5    12  30  2016   

                                        Country Name  
0                                      United States  
1                                               None  
2                                              Tonga  

In [52]:
joined = ee_data.merge(pop_dense_data,
          how='left',
          on =['Country Name', 'Year'],
         )
joined['Population'] = joined['Population'].fillna(0).astype(int)
# joined['Density'] = joined['Density'].fillna(0).astype(int)
print(joined)


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat